In [2]:
import numpy as np
from numpy import linalg
from scipy.special import binom
from collections import namedtuple

In [6]:
def calculate_n(row_index, col_index, num_points, degree):
    return (binom(num_points - degree, col_index) *
            sum((-1) ** (degree - k) * binom(degree, k) *
                binom(num_points - degree, row_index - k) / binom(2 * (num_points - degree), row_index + col_index - k)
                for k in range(degree+1)))

def calculate_n2(row_index, col_index, num_points, degree):
    sum_ = 0
    for k in range(degree+1):
        n1 = num_points - degree
        k1 = row_index - k
        n2 = 2 * (num_points - degree)
        k2 = row_index + col_index - k
        if ((n1 >= 0 and 0 <= k1 <= n1) and (n2 >= 0 and 0 <= k2 <= n2)):
            sum_ += (-1) ** (degree - k) * binom(degree, k) * binom(n1, k1) / binom(n2, k2)
    return sum_

In [23]:
def construct_n1_matrix(n):
    n_matrix = np.zeros((n-1, n-1))
    for i in range(1, n):
        for j in range(n-1):
            n_matrix[i-1][j] = calculate_n(i, j, n, 1) - calculate_n(i, j+1, n, 1)
    return n_matrix

def construct_n2_matrix(n):
    n_matrix = np.zeros((n-1, n-1))
    for i in range(1, n):
        n_matrix[i-1][0] = calculate_n2(i, 1, n, 2) - 2*calculate_n2(i, 0, n, 2)
        for j in range(n-3):
            n_matrix[i-1][j+1] = calculate_n2(i, j, n, 2) - 2*calculate_n2(i, j+1, n, 2) + calculate_n2(i, j+2, n, 2)
        n_matrix[i-1][n-2] = calculate_n2(i, n-3, n, 2) - 2*calculate_n2(i, 0, n-2, 2)
    return n_matrix

def construct_n3_matrix(n):
    n_matrix = np.zeros((n-1, n-1))
    for i in range(1, n):
        n_matrix[i-1][0] = 3*calculate_n2(i, 0, n, 3) - calculate_n2(i, 1, n, 3)
        n_matrix[i-1][1] = -3*calculate_n2(i, 0, n, 3) + 3*calculate_n2(i, 1, n, 3) - calculate_n2(i, 2, n, 3)
        for j in range(n-5):
            n_matrix[i-1][j+3] = calculate_n2(i, j, n, 3) - 3*calculate_n2(i, j+1, n, 3) + 3*calculate_n2(i, j+2, n, 3) - calculate_n2(i, j+3, n, 3)
        n_matrix[i-1][n-2] = calculate_n2(i, n-3, n, 2) - 2*calculate_n2(i, 0, n-2, 2)
    return n_matrix
        
def construct_b1_matrix(n, p0, pn):
    b = np.zeros((n-1, len(p0)))
    for i in range(1, n):
        b[i-1] = calculate_n(i, 0, n, 1)*p0 - calculate_n(i, n-1, n, 1)*pn
    return b


In [24]:
construct_n2_matrix(4)

array([[ 2.83333333, -1.        ,  3.83333333],
       [-0.5       ,  0.66666667, -2.16666667],
       [-0.5       , -1.        , -0.16666667]])

In [25]:
def add_known_point(a, b, p, p_index):
    a_new = np.delete(a, p_index-1, 0)
    b_new = np.delete(b, p_index-1, 0)
    b_new = b_new - a_new[:, [p_index-1]] * (p * np.ones((len(a_new), 1)))
    a_new = np.delete(a_new, p_index-1, 1)
    return a_new, b_new

In [26]:
def remove_cross_index_from_matrix(matrix, index):
    matrix_new = np.delete(matrix, index, 0)
    matrix_new = np.delete(matrix_new, index, 1)
    return matrix_new


In [29]:
from itertools import permutations

def calc_all_conds(matrix):
    conds = []
    n = len(matrix)
    conds.append(linalg.cond(matrix))
    if n > 2:
        possible_indices = range(n)        
        for index_order in permutations(list(possible_indices)):
            for index in index_order:
                matrix_new = remove_cross_index_from_matrix(matrix, index)
                conds.extend(calc_all_conds(matrix_new))
    return conds

def get_conds_n1():
    return [linalg.cond(construct_n2_matrix(n)) for n in range(3, 30)]

get_conds_n1()

[5.8284271247461916,
 8.2568599252237096,
 79.632928101567288,
 271.02096954690535,
 1078.7690135021176,
 6477.8701955151691,
 42727.53533750747,
 279645.74201669829,
 1906963.3451004068,
 13378746.852464506,
 95500381.397996634,
 690792228.75903881,
 5046307782.4594698,
 37166193531.411469,
 275533049462.58405,
 2054121732120.5261,
 15386850157917.654,
 115738126473884.42,
 873746428300204.87,
 6617656450870382.0,
 50264448322934944.0,
 3.8277938762421664e+17,
 2.9195496506335288e+18,
 2.2289424791595217e+19,
 1.6782350336025556e+20,
 1.2492538503109254e+21,
 7.510479334792363e+21]

In [216]:
a = construct_n1_matrix(4)
print(linalg.cond(a, np.inf))
print('a: {}'.format(a))
a
b = construct_b_matrix(4, np.array([1,2]), np.array([2,3]))
print('b: {}'.format(b))
p = np.array([3, 4])
p_index = 3
a_new, b_new = add_known_point(a, b, p, p_index)
a = a_new
b = b_new
print('after')
print('a_new: {}'.format(a))
print('b_new: {}'.format(b))
print(linalg.cond(a, np.inf))

{'num_points': 4, 'row_index': 1, 'col_index': 0, 'degree': 1}
1.0
K: 0
(1 0) = 1.0
(3 1) = 3.0
(6 1) = 6.0
K: 1
(1 1) = 1.0
(3 0) = 1.0
(6 0) = 1.0
{'num_points': 4, 'row_index': 1, 'col_index': 1, 'degree': 1}
3.0
K: 0
(1 0) = 1.0
(3 1) = 3.0
(6 2) = 15.0
K: 1
(1 1) = 1.0
(3 0) = 1.0
(6 1) = 6.0
{'num_points': 4, 'row_index': 1, 'col_index': 1, 'degree': 1}
3.0
K: 0
(1 0) = 1.0
(3 1) = 3.0
(6 2) = 15.0
K: 1
(1 1) = 1.0
(3 0) = 1.0
(6 1) = 6.0
{'num_points': 4, 'row_index': 1, 'col_index': 2, 'degree': 1}
3.0
K: 0
(1 0) = 1.0
(3 1) = 3.0
(6 3) = 20.0
K: 1
(1 1) = 1.0
(3 0) = 1.0
(6 2) = 15.0
{'num_points': 4, 'row_index': 1, 'col_index': 2, 'degree': 1}
3.0
K: 0
(1 0) = 1.0
(3 1) = 3.0
(6 3) = 20.0
K: 1
(1 1) = 1.0
(3 0) = 1.0
(6 2) = 15.0
{'num_points': 4, 'row_index': 1, 'col_index': 3, 'degree': 1}
1.0
K: 0
(1 0) = 1.0
(3 1) = 3.0
(6 4) = 15.0
K: 1
(1 1) = 1.0
(3 0) = 1.0
(6 3) = 20.0
{'num_points': 4, 'row_index': 2, 'col_index': 0, 'degree': 1}
1.0
K: 0
(1 0) = 1.0
(3 2) = 3.0
(6

In [1]:
list(range(5, 4))


[]